# Tavus API Integration Guide

This notebook provides a step-by-step guide for integrating with the Tavus API, including:

1. Getting Replicas
2. Creating a Persona
3. Creating a Conversation
4. Webhook Setup and Explanation

---


## Setup

First, let's set up our environment and import necessary libraries.


In [6]:
import requests
import json
import os
from typing import Dict, List, Optional
from datetime import datetime


In [8]:
# Configuration
TAVUS_API_KEY = os.getenv('VITE_TAVUS_API_KEY', 'your-api-key-here')
TAVUS_BASE_URL = 'https://tavusapi.com/v2'

# Headers for API requests
headers = {
    'x-api-key': TAVUS_API_KEY,
    'Content-Type': 'application/json'
}

print(f"API Base URL: {TAVUS_BASE_URL}")
print(f"API Key configured: {'Yes' if TAVUS_API_KEY != 'your-api-key-here' else 'No - Please set TAVUS_API_KEY environment variable'}")


API Base URL: https://tavusapi.com/v2
API Key configured: Yes


---

## Step 1: Getting Replicas

Replicas are AI video replicas that can be used in conversations. Let's fetch all available replicas.


In [10]:
def get_replicas() -> List[Dict]:
    """
    Fetches all available replicas from Tavus API.
    
    Returns:
        List of replica objects containing replica_id, replica_name, etc.
    """
    try:
        response = requests.get(
            f"{TAVUS_BASE_URL}/replicas",
            headers=headers
        )
        
        response.raise_for_status()
        data = response.json()
        
        # Tavus API returns data in a 'data' field
        replicas = data.get('data', [])
        
        print(f"✅ Successfully fetched {len(replicas)} replica(s)")
        return replicas
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching replicas: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")
        return []


In [11]:
# Fetch replicas
replicas = get_replicas()

# Display replicas
if replicas:
    print("\n📋 Available Replicas:")
    print("=" * 60)
    for i, replica in enumerate(replicas, 1):
        print(f"\n{i}. Replica ID: {replica.get('replica_id', 'N/A')}")
        print(f"   Name: {replica.get('replica_name', 'N/A')}")
        print(f"   Status: {replica.get('status', 'N/A')}")
        if replica.get('created_at'):
            print(f"   Created: {replica.get('created_at')}")
else:
    print("\n⚠️  No replicas found. Make sure you have created replicas in your Tavus account.")


✅ Successfully fetched 53 replica(s)

📋 Available Replicas:

1. Replica ID: rfcc944ac6
   Name: Steph - Selfie (Deprecated, Phoenix-1)
   Status: completed
   Created: 2024-07-19T14:20:19.834Z

2. Replica ID: r9d30b0e55ac
   Name: Luna
   Status: completed
   Created: 2025-03-31T10:53:42.843Z

3. Replica ID: rf4703150052
   Name: Charlie
   Status: completed
   Created: 2025-03-31T10:56:11.882Z

4. Replica ID: rc2146c13e81
   Name: Olivia
   Status: completed
   Created: 2025-03-31T10:57:11.457Z

5. Replica ID: r4317e64d25a
   Name: Gloria
   Status: completed
   Created: 2025-03-31T10:58:03.486Z

6. Replica ID: r6ae5b6efc9d
   Name: Anna
   Status: completed
   Created: 2025-03-31T10:59:00.437Z

7. Replica ID: r9c55f9312fb
   Name: Steph - Office V1
   Status: completed
   Created: 2025-03-31T10:59:46.828Z

8. Replica ID: r1a4e22fa0d9
   Name: Benjamin
   Status: completed
   Created: 2025-03-31T11:17:38.857Z

9. Replica ID: r68fe8906e53
   Name: Mary - Office
   Status: completed
   

In [12]:
# Store the first replica ID for later use
if replicas:
    selected_replica_id = replicas[0].get('replica_id')
    print(f"\n✅ Selected Replica ID: {selected_replica_id}")
    print("   (We'll use this for creating personas and conversations)")
else:
    selected_replica_id = None
    print("\n⚠️  No replica selected. Please create a replica first.")



✅ Selected Replica ID: rfcc944ac6
   (We'll use this for creating personas and conversations)


---

## Step 2: Creating a Persona

A persona defines the behavior and personality of the AI in conversations. It includes a system prompt that guides how the AI responds.


In [13]:
def create_persona(
    system_prompt: str,
    pipeline_mode: str = 'full',
    persona_name: Optional[str] = None
) -> Optional[Dict]:
    """
    Creates a new persona in Tavus.
    
    Args:
        system_prompt: The system prompt that defines the persona's behavior
        pipeline_mode: Pipeline mode (default: 'full')
        persona_name: Optional name for the persona
    
    Returns:
        Persona object with persona_id and other details
    """
    try:
        payload = {
            'pipeline_mode': pipeline_mode,
            'system_prompt': system_prompt
        }
        
        if persona_name:
            payload['persona_name'] = persona_name
        
        response = requests.post(
            f"{TAVUS_BASE_URL}/personas",
            headers=headers,
            json=payload
        )
        
        response.raise_for_status()
        persona = response.json()
        
        print(f"✅ Successfully created persona")
        print(f"   Persona ID: {persona.get('persona_id', 'N/A')}")
        
        return persona
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creating persona: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")
        return None


In [14]:
# Example: Create a Sales Engineer Hiring Persona
sales_engineer_prompt = """
As a Sales Engineer hiring manager, you are conducting a technical interview for a Sales Engineer position.

Your role is to:
- Assess the candidate's technical knowledge and problem-solving abilities
- Evaluate their communication skills and ability to explain complex concepts
- Understand their experience with customer-facing technical roles
- Determine their fit for the Sales Engineer role

Ask thoughtful questions about:
1. Technical background and experience
2. Experience working with customers or clients
3. Problem-solving approaches
4. Communication and presentation skills

Be professional, engaging, and create a comfortable interview environment.
Provide constructive feedback when appropriate.
"""

persona = create_persona(
    system_prompt=sales_engineer_prompt,
    pipeline_mode='full',
    persona_name='Sales Engineer Hiring Manager'
)


✅ Successfully created persona
   Persona ID: p4c505242243


In [15]:
# Store persona ID for creating conversations
if persona:
    selected_persona_id = persona.get('persona_id')
    print(f"\n✅ Selected Persona ID: {selected_persona_id}")
    print("   (We'll use this for creating conversations)")
else:
    selected_persona_id = None
    print("\n⚠️  No persona created. Please check the error above.")



✅ Selected Persona ID: p4c505242243
   (We'll use this for creating conversations)


---

## Step 3: Creating a Conversation

A conversation is an interactive session between a user and the AI persona. You need both a replica and a persona to create a conversation.


In [16]:
def create_conversation(
    replica_id: str,
    persona_id: str,
    conversation_name: Optional[str] = None
) -> Optional[Dict]:
    """
    Creates a new conversation session.
    
    Args:
        replica_id: The ID of the replica to use
        persona_id: The ID of the persona to use
        conversation_name: Optional name for the conversation
    
    Returns:
        Conversation object with conversation_id, conversation_url, etc.
    """
    try:
        payload = {
            'replica_id': replica_id,
            'persona_id': persona_id
        }
        
        if conversation_name:
            payload['conversation_name'] = conversation_name
        
        response = requests.post(
            f"{TAVUS_BASE_URL}/conversations",
            headers=headers,
            json=payload
        )
        
        response.raise_for_status()
        conversation = response.json()
        
        print(f"✅ Successfully created conversation")
        print(f"   Conversation ID: {conversation.get('conversation_id', 'N/A')}")
        print(f"   Conversation URL: {conversation.get('conversation_url', 'N/A')}")
        
        return conversation
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creating conversation: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response: {e.response.text}")
        return None


In [17]:
# Create a conversation using the selected replica and persona
if selected_replica_id and selected_persona_id:
    conversation = create_conversation(
        replica_id=selected_replica_id,
        persona_id=selected_persona_id,
        conversation_name=f"Sales Engineer Interview - {datetime.now().strftime('%Y-%m-%d %H:%M')}"
    )
    
    if conversation:
        conversation_id = conversation.get('conversation_id')
        conversation_url = conversation.get('conversation_url')
        print(f"\n🎉 Conversation created successfully!")
        print(f"\n📝 Next steps:")
        print(f"   1. Share this URL with the candidate: {conversation_url}")
        print(f"   2. The candidate can join the conversation using this URL")
        print(f"   3. After the conversation ends, use webhooks to get the transcript and analysis")
else:
    print("\n⚠️  Cannot create conversation. Please ensure you have:")
    print("   - At least one replica")
    print("   - At least one persona")


❌ Error creating conversation: 402 Client Error: PAYMENT REQUIRED for url: https://tavusapi.com/v2/conversations
Response: {"message":"The user is out of conversational credits."}



---

## Step 4: Webhook Setup and Explanation

Webhooks allow Tavus to notify your application when events occur (e.g., when a conversation ends, transcript is ready, analysis is complete).


### 4.1 Understanding Webhooks

**What are Webhooks?**

Webhooks are HTTP callbacks that allow Tavus to send real-time notifications to your application when specific events occur. Instead of polling the API repeatedly, webhooks push data to your server when events happen.

**Common Webhook Events:**

1. **Conversation Started** - When a participant joins
2. **Conversation Ended** - When the conversation is completed
3. **Transcript Ready** - When the transcript is available
4. **Analysis Ready** - When the conversation analysis is complete

**Webhook Flow:**

```
┌─────────────┐         ┌──────────────┐         ┌─────────────┐
│   Tavus     │────────▶│   Your       │────────▶│   Your       │
│   Platform  │  POST   │   Webhook    │  Process│   Database   │
│             │  Event  │   Endpoint   │  Data   │   / Storage  │
└─────────────┘         └──────────────┘         └─────────────┘
```

1. Event occurs in Tavus (e.g., conversation ends)
2. Tavus sends HTTP POST request to your webhook URL
3. Your server receives and processes the webhook payload
4. You store/process the data as needed


### 4.2 Webhook Payload Structure

When Tavus sends a webhook, it includes information about the event. Here's an example payload:


In [ ]:
# Example webhook payload structure
example_webhook_payload = {
    "event_type": "conversation.ended",  # Type of event
    "conversation_id": "c46aee906d1d640c",  # ID of the conversation
    "timestamp": "2025-12-05T21:21:41.031Z",  # When the event occurred
    "data": {
        "conversation_id": "c46aee906d1d640c",
        "replica_id_used": "rfe12d8b9597",
        "shutdown_reason": "participant_left_timeout",
        "shutdown_timestamp": "2025-12-05T21:21:41.031Z",
        # Additional data depending on event type
    }
}

print("Example Webhook Payload:")
print(json.dumps(example_webhook_payload, indent=2))


### 4.3 Setting Up Webhooks

**Step 1: Create a Webhook Endpoint**

You need to create an HTTP endpoint that can receive POST requests from Tavus. This can be:

- A serverless function (AWS Lambda, Vercel, Netlify)
- A webhook service (n8n, Zapier, Make.com)
- Your own server endpoint

**Step 2: Configure Webhook in Tavus Dashboard**

1. Go to your Tavus dashboard
2. Navigate to Settings → Webhooks
3. Add a new webhook URL
4. Select which events you want to receive
5. Save the configuration

**Step 3: Verify Webhook (Optional)**

Tavus may send a verification request to ensure your endpoint is valid.


### 4.4 Example Webhook Handler (Python/Flask)


In [ ]:
# Example Flask webhook handler (for reference - not executable in notebook)
webhook_handler_example = """
from flask import Flask, request, jsonify
import json

app = Flask(__name__)

@app.route('/webhook/tavus', methods=['POST'])
def tavus_webhook():
    try:
        # Get the webhook payload
        payload = request.get_json()
        
        # Extract event information
        event_type = payload.get('event_type')
        conversation_id = payload.get('conversation_id')
        event_data = payload.get('data', {})
        
        # Handle different event types
        if event_type == 'conversation.ended':
            print(f"Conversation {conversation_id} ended")
            # Process conversation end event
            # e.g., fetch transcript, trigger analysis, etc.
            
        elif event_type == 'transcript.ready':
            print(f"Transcript ready for conversation {conversation_id}")
            # Process transcript
            transcript = event_data.get('transcript')
            # Store transcript in database
            
        elif event_type == 'analysis.ready':
            print(f"Analysis ready for conversation {conversation_id}")
            # Process analysis
            analysis = event_data.get('analysis')
            # Store analysis in database
        
        # Return success response
        return jsonify({'status': 'success'}), 200
        
    except Exception as e:
        print(f"Error processing webhook: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

if __name__ == '__main__':
    app.run(port=5000)
"""

print("Example Webhook Handler:")
print(webhook_handler_example)


### 4.5 Using Webhook Services (n8n, Zapier, Make.com)

**Using Automation Platforms:**

You can use webhook automation platforms to process Tavus webhooks without writing server code:

**n8n Setup (Example):**

1. **Create Webhook Node:**
   - Add a "Webhook" node
   - Set HTTP Method to POST
   - Copy the webhook URL
   - Configure in Tavus dashboard

2. **Process Webhook Data:**
   - Add a "Function" node to parse the payload
   - Extract `conversation_id` from the webhook
   - Use it to fetch conversation details via Tavus API

3. **Store Data:**
   - Add a database node (PostgreSQL, MySQL, etc.)
   - Store transcript, analysis, and metadata

**Example Workflow:**

```
Tavus Webhook → Parse Payload → Fetch Details from Tavus API → Store in Database
```

**Other Options:**
- **Zapier**: Create a Zap with webhook trigger
- **Make.com**: Use webhook module to receive events
- **Custom Server**: Build your own endpoint (see Flask example above)


In [ ]:
# Example: Fetch conversation details directly from Tavus API
# (This would typically be called from your webhook handler after receiving a webhook event)

def get_conversation_details_from_api(conversation_id: str) -> Optional[Dict]:
    """
    Gets conversation details directly from Tavus API.
    
    Note: This is an example of how you might fetch conversation details
    after receiving a webhook event. In practice, you would call this
    from your webhook handler when you receive a 'conversation.ended' event.
    
    Note: Tavus API may require additional endpoints or authentication
    for fetching conversation details. Check Tavus documentation for the
    correct endpoint.
    """
    try:
        # Example: Fetch conversation details
        # Note: Replace with actual Tavus API endpoint for getting conversation details
        response = requests.get(
            f"{TAVUS_BASE_URL}/conversations/{conversation_id}",
            headers=headers
        )
        
        response.raise_for_status()
        data = response.json()
        
        print(f"✅ Successfully fetched conversation details")
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching conversation details: {e}")
        print("Note: The actual endpoint may differ. Check Tavus API documentation.")
        return None


In [ ]:
# Example: Get conversation details after conversation ends
# (This would typically be triggered by a webhook event, but we're calling it manually here)
# In a real scenario, your webhook handler would call this function when it receives
# a 'conversation.ended' event from Tavus.

if 'conversation_id' in locals():
    print(f"Fetching details for conversation: {conversation_id}")
    print("Note: In production, this would be called from your webhook handler.")
    print("      The webhook would provide the conversation_id automatically.\n")
    
    # Example: Fetch details (replace with actual implementation)
    # details = get_conversation_details_from_api(conversation_id)
    
    print("📝 Webhook Handler Flow:")
    print("   1. Tavus sends webhook → Your endpoint receives POST request")
    print("   2. Extract conversation_id from webhook payload")
    print("   3. Call Tavus API to fetch full conversation details")
    print("   4. Process and store transcript, analysis, and metadata")
    print("   5. Return HTTP 200 to acknowledge receipt")
else:
    print("⚠️  No conversation_id available. Create a conversation first.")
    print("   Once you have a conversation_id, you can test fetching its details.")


### 4.6 Webhook Action Breakdown

**How Webhook Actions Work:**

1. **Event Occurs** → Tavus detects an event (e.g., conversation ends)

2. **Webhook Triggered** → Tavus sends HTTP POST to your webhook URL
   ```
   POST https://your-webhook-url.com/tavus
   Content-Type: application/json
   
   {
     "event_type": "conversation.ended",
     "conversation_id": "c46aee906d1d640c",
     "timestamp": "2025-12-05T21:21:41.031Z",
     "data": {...}
   }
   ```

3. **Your Server Receives** → Your webhook endpoint receives the POST request

4. **Process Event** → Your code:
   - Validates the request (optional: verify signature)
   - Extracts `conversation_id` from the payload
   - Determines action based on `event_type`

5. **Fetch Additional Data** (if needed):
   - Call Tavus API to get full conversation details
   - Use the `conversation_id` to retrieve transcript and analysis
   - Process the data as needed

6. **Store/Process Data**:
   - Save transcript to database
   - Store analysis results
   - Update conversation status
   - Trigger notifications or other workflows

7. **Return Response** → Your server returns HTTP 200 to acknowledge receipt

**Important Notes:**

- Webhooks should respond quickly (< 5 seconds)
- For long-running tasks, queue them and process asynchronously
- Always return HTTP 200 even if processing fails (to prevent retries)
- Log webhook events for debugging
- Consider webhook signature verification for security
- Handle errors gracefully and implement retry logic if needed


### 4.6 Webhook Action Breakdown

**How Webhook Actions Work:**

1. **Event Occurs** → Tavus detects an event (e.g., conversation ends)

2. **Webhook Triggered** → Tavus sends HTTP POST to your webhook URL
   ```
   POST https://your-webhook-url.com/tavus
   Content-Type: application/json
   
   {
     "event_type": "conversation.ended",
     "conversation_id": "c46aee906d1d640c",
     "timestamp": "2025-12-05T21:21:41.031Z",
     "data": {...}
   }
   ```

3. **Your Server Receives** → Your webhook endpoint receives the POST request

4. **Process Event** → Your code:
   - Validates the request (optional: verify signature)
   - Extracts `conversation_id`
   - Determines action based on `event_type`

5. **Fetch Additional Data** (if needed):
   - Call Tavus API to get full conversation details
   - Or use your n8n webhook to get processed data

6. **Store/Process Data**:
   - Save transcript to database
   - Store analysis results
   - Update conversation status
   - Trigger notifications or other workflows

7. **Return Response** → Your server returns HTTP 200 to acknowledge receipt

**Important Notes:**

- Webhooks should respond quickly (< 5 seconds)
- For long-running tasks, queue them and process asynchronously
- Always return HTTP 200 even if processing fails (to prevent retries)
- Log webhook events for debugging
- Consider webhook signature verification for security


### 4.7 Complete Workflow Example

Here's how the complete flow works:


In [ ]:
# Complete workflow visualization
workflow_diagram = """
┌─────────────────────────────────────────────────────────────────┐
│                    COMPLETE TAVUS WORKFLOW                       │
└─────────────────────────────────────────────────────────────────┘

STEP 1: Setup
├─ Get Replicas (GET /v2/replicas)
├─ Create Persona (POST /v2/personas)
└─ Store IDs for later use

STEP 2: Create Conversation
├─ Create Conversation (POST /v2/conversations)
│  ├─ replica_id: <from step 1>
│  └─ persona_id: <from step 1>
├─ Receive conversation_id and conversation_url
└─ Share URL with participant

STEP 3: Conversation Happens
├─ Participant joins via conversation_url
├─ AI persona interacts with participant
└─ Conversation ends (participant leaves or timeout)

STEP 4: Webhook Processing (Automatic)
├─ Tavus sends webhook to your endpoint
│  └─ Event: conversation.ended
│  └─ Payload: {conversation_id, event_type, data}
├─ Your webhook receives POST request
├─ Extract conversation_id
├─ Fetch conversation details from Tavus API
├─ Process transcript and analysis
└─ Store in database

STEP 5: Retrieve Data
├─ List Conversations (GET /v2/conversations)
├─ Get Conversation Details (via webhook or API)
└─ Display transcript and analysis
"""

print(workflow_diagram)


---

## Summary

This notebook covered:

1. ✅ **Getting Replicas** - Fetch available AI video replicas
2. ✅ **Creating Personas** - Define AI behavior with system prompts
3. ✅ **Creating Conversations** - Start interactive sessions
4. ✅ **Webhook Setup** - Understand how webhooks work and how to set them up

**Next Steps:**

- Set up your webhook endpoint
- Configure webhooks in Tavus dashboard
- Test the complete workflow
- Integrate with your application

**Resources:**

- Tavus API Documentation: https://docs.tavus.io
- API Base URL: https://tavusapi.com/v2
- Webhook Setup: Configure in your Tavus dashboard under Settings → Webhooks
